In [1]:
import torch   
from torchtext import data 

In [2]:
#Reproducing same results
SEED = 2019

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True  

In [3]:
tokenizer = lambda x: x.split()
TEXT = data.Field(tokenize=tokenizer,batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)

In [4]:
fields = [('text',TEXT),('label', LABEL)]

In [6]:
import pandas as pd
df = pd.read_csv("Mortality.csv",
                     header=0)
print(df.shape)
print(df.head())

(1855, 2)
                                                text  label
0  admission date                discharge date  ...      0
1  admission date       discharge date    date of...      0
2  admission date                discharge date  ...      0
3  admission date                discharge date  ...      0
4  admission date                discharge date  ...      0


In [10]:
#loading custom dataset
path = 'Mortality.csv'
fields = [('text',TEXT),('label', LABEL)]
training_data=data.TabularDataset(path = path,format = 'csv',fields = fields, skip_header = True)

#print preprocessed text
#print(vars(training_data.examples[1000]))

### Preparing input and output sequences

In [11]:
import random
train_data, valid_data = training_data.split(split_ratio=0.7, stratified = True, random_state = random.seed(SEED))

In [222]:
#initialize glove embeddings
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi)  

Size of TEXT vocabulary: 11699
Size of LABEL vocabulary: 2
[('the', 56471), ('and', 46215), ('to', 40353), ('of', 38222), ('was', 33632), ('a', 23837), ('with', 23553), ('on', 22428), ('in', 20812), ('no', 19207)]
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fde11bf9a20>>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'and': 3, 'to': 4, 'of': 5, 'was': 6, 'a': 7, 'with': 8, 'on': 9, 'in': 10, 'no': 11, 'for': 12, 'is': 13, 'name': 14, 'he': 15, 'mg': 16, 'left': 17, 'or': 18, 'she': 19, 'right': 20, 'patient': 21, 'at': 22, 'as': 23, 'blood': 24, 'hospital': 25, 'discharge': 26, 'tablet': 27, 'his': 28, 'po': 29, 'her': 30, 'hemorrhage': 31, 'not': 32, 'ct': 33, 'history': 34, 'there': 35, 'sig': 36, 'am': 37, 'be': 38, 'last': 39, 'had': 40, 'were': 41, 'this': 42, 'one': 43, 'daily': 44, 'head': 45, 'are': 46, 'an': 47, 'day': 48, 'you': 49, 'that': 50, 'pm': 51, 'by': 52, 'but': 53, 'your': 54, 'date': 55, 'normal': 56, 'from': 57, 'admission': 58, 

In [225]:
#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 64

#Load an iterator
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = device)

### Prediction Model

In [227]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

In [228]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 32
num_output_nodes = 1
num_layers = 2
bidirection = True
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout)

In [229]:
#architecture
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

classifier(
  (embedding): Embedding(11699, 100)
  (lstm): LSTM(100, 32, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (act): Sigmoid()
)
The model has 1,229,357 trainable parameters
torch.Size([11699, 100])


In [230]:
import torch.optim as optim
from sklearn.metrics import classification_report

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [231]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.text   
        
        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()  
        
        #compute the loss
        loss = criterion(predictions, batch.label)        
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [232]:
import numpy as np

def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            #retrieve text and no. of words
            text, text_lengths = batch.text
            
            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()
            
            #compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
            pred_labels = [1 if i > 0.5 else 0 for i in predictions.cpu().numpy()]

            
            report = classification_report(batch.label.cpu().numpy(), pred_labels)
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), report

In [233]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc,report = evaluate(model, valid_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print(report)

/home/ran/miniconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


	Train Loss: 0.592 | Train Acc: 75.07%
	 Val. Loss: 0.551 |  Val. Acc: 74.94%
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96        41
         1.0       0.00      0.00      0.00         3

    accuracy                           0.93        44
   macro avg       0.47      0.50      0.48        44
weighted avg       0.87      0.93      0.90        44

	Train Loss: 0.540 | Train Acc: 75.07%
	 Val. Loss: 0.503 |  Val. Acc: 74.94%
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96        41
         1.0       0.00      0.00      0.00         3

    accuracy                           0.93        44
   macro avg       0.47      0.50      0.48        44
weighted avg       0.87      0.93      0.90        44

	Train Loss: 0.429 | Train Acc: 77.01%
	 Val. Loss: 0.348 |  Val. Acc: 91.24%
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99        41
   